In [115]:
import pandas as pd 
import numpy as np
import spacy
from spacy import displacy
from spacy.lang.en import English
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

!python -m spacy download en_core-web_sm


✘ No compatible package found for 'en_core-web_sm' (spaCy v3.8.7)



In [116]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

In [117]:
with open('20th_century_article_Wiki.txt', 'r', errors='ignore') as file: 
   data = file.read().replace( '\n', ' ')

wiki = NER(data)

# Visualize identified entities

displacy.render(wiki[273:20000], style = "ent", jupyter = True)

In [118]:
# Loop through sentences, get entity list for each sentence
wiki_hold = []
for sent in wiki.sents:
       entity_list = [ent.text for ent in sent.ents]
       wiki_hold.append({'sentence': sent, 'entities': entity_list})

df_sentences = pd.DataFrame(wiki_hold)

In [119]:
df_sentences.head(10)

,sentence,entities
0,"( , Key, events, of, the, 20th, century, -,...",[the 20th century]
1,"(articleAbout, WikipediaContact, us, \t\t...",[Search Search ...
2,"(The, rise, of, dictatorship, , 1.4,...","[1.4, World War II, 1.4.1]"
3,"(The, war, in, Europe, , 1.4.2, Blitzk...","[Europe, 1.4.2, Blitzkrieg 1.4.3, Oper..."
4,"(Turning, tides, )",[]
5,"(1.4.5, Operation, Overlord, , 1.4.6, ...",[Operation Overlord 1.4.6 Final days ...
6,"(Allied, offensive, , 1.4.10, Final, d...",[Allied offensive 1.4.10 Final days ...
7,"(The, Holocaust, , 1.4.12, The, Nuclea...","[The Nuclear Age, 1.5]"
8,"(The, post, -, war, world, , 1.5.1)",[1.5.1]
9,"(The, end, of, empires, :, decolonization, ...","[The Cold War, 1947â€“1991]"


In [120]:
csv_filepath = r"C:\Users\stefa\20th-century\countries_list_20th_century_1.5.csv"

In [121]:
df = pd.read_csv(csv_filepath)

In [122]:
df = df.drop('Unnamed: 0', axis=1)
df

,country_name
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
204,Sahrawi Arab Democratic Republic
205,Somaliland
206,South Ossetia
207,Taiwan


In [123]:
#df['country_name'] = df['country_name'].str.replace()
#df['country_name'] = df['country_name'].str.replace(r'\s+', '', regex=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_new = df
print(df_new)

                         country_name
0                         Afghanistan
1                             Albania
2                             Algeria
3                             Andorra
4                              Angola
..                                ...
204  Sahrawi Arab Democratic Republic
205                        Somaliland
206                     South Ossetia
207                            Taiwan
208                      Transnistria

[209 rows x 1 columns]


C:\Users\stefa\AppData\Local\Temp\ipykernel_23708\2503647563.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [124]:
def filter_entity(ent_list, df_new):
       return [ent for ent in ent_list
                  if ent in list(df_new['country_name'])]

In [125]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, df_new))

In [126]:
# Filter out sentences that don’t have any character entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

df_sentences_filtered.tail(10)

,sentence,entities,country_entities
1117,"("", The, forgotten, violence, that, helped, In...",[India],[India]
1122,"("", Indian, Independence, Day, :, everything, ...","[Indian Independence Day, Partition between In...",[Pakistan]
1131,"(^, "", The, Philippines, ,, 1898â€“1946, |, US...","[Philippines, 1898â€“1946, US House of Represe...",[Philippines]
1161,"("", Colonial, Cartographies, ,, Postcolonial, ...","[Colonial Cartographies, Postcolonial Borders,...",[Afghanistan]
1199,"(The, Moldovans, :, Romania, ,, Russia, ,, and...","[Moldovans, Romania, Russia, the Politics of C...","[Romania, Russia]"
1263,"("", Selling, ', Operation, Passage, to, Freedo...","[Thomas Dooley, the Religious Overtones of Ear...",[Vietnam]
1294,"("", Stuck, in, Endless, Preliminaries, :, Viet...","[Stuck in Endless Preliminaries, Vietnam, the ...",[Vietnam]
1569,"("", Anti, -, American, Behavior, in, the, Midd...","[Anti-American Behavior, the Middle East, a Fi...",[Lebanon]
1575,"(The, Rise, of, China, and, India, :, A, New, ...","[India, New Asian]",[India]
1576,"(Singapore, :, World, Scientific, .)","[Singapore, World Scientific]",[Singapore]


In [ ]:
# Defining relationships

# window size = 5 : this defines how many sentences will be looked at simultaneously
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
     end_i = min(i+5, df_sentences_filtered.index[-1])
     country_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])

     # Remove duplicated characters that are next to each other
     country_unique = [country_list[i] for i in range(len(country_list))
                             if (i==0) or country_list[i] != country_list[i-1]]

if len(country_unique) > 1:
    for idx, a in enumerate(country_unique[ :-1]):
          b = country_unique[idx + 1]
          relationships.append({"source": a, " target": b})

In [128]:
relationship_df = pd.DataFrame(relationships)

relationship_df

,source,target
0,India,Singapore
